In [ ]:
import os
import sys

# Change directory check to match the new folder name
ENDWITHS = 'NTREX'

NOTEBOOK_DIR = os.getcwd()
print(f"Current working directory: {NOTEBOOK_DIR}")
print(f"Ends with '{ENDWITHS}': {NOTEBOOK_DIR.endswith(ENDWITHS)}")

In [ ]:
if not NOTEBOOK_DIR.endswith(ENDWITHS):
    # Define the path to NTREX directory
    # Assuming standard project structure
    NTREX_PATH = os.path.join(NOTEBOOK_DIR, 'code', 'pipeline', 'Evaluators', 'Translators', 'NTREX')
    
    # Check if NTREX directory exists
    if os.path.exists(NTREX_PATH):
        print(f"\nChanging directory to: {NTREX_PATH}")
        os.chdir(NTREX_PATH)
        NOTEBOOK_DIR = os.getcwd()
        print(f"New working directory: {NOTEBOOK_DIR}")
        print(f"Now ends with '{ENDWITHS}': {NOTEBOOK_DIR.endswith(ENDWITHS)}")
    else:
        # Try to create it or find alternative path
        print(f"\nNTREX directory not found at: {NTREX_PATH}")
        print("Searching for NTREX directory...")
        
        # Search recursively
        for root, dirs, files in os.walk(NOTEBOOK_DIR):
            if 'NTREX' in dirs:
                ntrex_dir = os.path.join(root, 'NTREX')
                print(f"Found NTREX at: {ntrex_dir}")
                os.chdir(ntrex_dir)
                NOTEBOOK_DIR = os.getcwd()
                print(f"Changed to: {NOTEBOOK_DIR}")
                break
        else:
            print("NTREX directory not found. Creating it...")

In [ ]:
if not NOTEBOOK_DIR.endswith(ENDWITHS):
    raise ValueError(f"Not in correct dir, expect end with {ENDWITHS}, but got {NOTEBOOK_DIR} instead")

# Locate the root directory to import modules from 'code' folder
BASE_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, '..', '..', '..', '..', '..'))
print(f"Base Directory: {BASE_DIR}")

# Add the 'code' directory to sys.path to allow imports like 'pipeline.TranslationModels...'
sys.path.insert(0, os.path.join(BASE_DIR, 'code'))

In [ ]:
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

In [ ]:
# Import the Evaluator class from the local .py file (NTREXEvaluator.py)
from NTREXEvaluator import NTREXEvaluator

# Import the Translation Model (Assuming this path exists in your project structure)
from pipeline.TranslationModels.ElanMtJaEnTranslator import ElanMtJaEnTranslator

In [ ]:
from dotenv import load_dotenv

# Load .env file from the project root (adjust path logic if necessary)
load_dotenv(os.path.abspath(os.path.join(BASE_DIR, '..', '.env')))

HF_TOKEN = os.getenv('HF_TOKEN')
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

# Verify token is loaded (Optional security check)
if HF_TOKEN:
    print("HuggingFace Token loaded successfully.")
else:
    print("Warning: HF_TOKEN not found in .env")

In [ ]:
# Initialize the Evaluator with the HF Token and Device
evaluator = NTREXEvaluator(hf_token=HF_TOKEN, device=device)

# Initialize the Translation Model
# Note: Ensure ElanMtJaEnTranslator class exists and works as expected
translation_model = ElanMtJaEnTranslator()

# Load the model weights (using 'base' or other available configurations)
translation_model.load_model(device=device, elan_model='base')

# Run evaluation
# Note: xianf/NTREX is a test set, so we expect 'split=test' logic inside the evaluator
metrics = evaluator.evaluate(translation_model, device=device, batch_size=1, verbose=False)